In [ ]:
# Bibliotecas Necessárias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from  matplotlib import animation, rc
from IPython.display import HTML
import csv

In [ ]:
# Variaveis globais para o comportamento das simulações

G = 9.8 # Aceleração da Gravidade
M = 1.0 # Massa do Objeto -- Se precisar 
dt = 0.01 # Variação de tempo
THETA = np.pi/4 # Ângulo para o disparo do projétil
LIMIT = 1000 # Numero de variações de estado máxima para a simulação

In [ ]:
# Funções para realizar as animações

def init():
    '''
        Define como o canvas deverá ser iniciliazado.
    '''
    line.set_data([],[])
    line2.set_data([], [])
    text.set_text('')
    return line, line2, text
    

def update(i, t_euler, t_analitica):
    '''
        Define o que será desenhado a cada frame
    '''
    line.set_data(*t_euler[i])
    line2.set_data(*t_analitica[i])
    text.set_text(text_template % (i*dt))
    return line, line2, text




def setUp (range_x, range_y, Titulo):
    fig, (ax) = plt.subplots(1, figsize=(10, 10))
    ax.set_xlim(*range_x)
    ax.set_ylim(*range_y)
    
    ax.set_title(Titulo)
    ax.grid()
    
    line, = ax.plot([],[], 'o-', lw=2, label='Euler')
    line2, = ax.plot([], [], 'o-', lw=2, label='Analítica')
    text_template = "Tempo: %.2fs"
    text = ax.text(0.05, 0.9, '', transform=ax.transAxes)     
        
    return fig, ax, line, line2, text_template, text    
        

# O que precisa ser (infelizmente) copiado em cada notebook:

# t_euler = main()
# t_analitica = trajetoria_analitica()
# range_x = (0, 15)
# range_y = (0, 4)


# fig, ax, line, line2, text_template, text = setUp(range_x, range_y, "NOME_SIMULAÇÃO")
    
# intervalo = dt*LIMIT
# animacao_"alguma_coisa" = animation.FuncAnimation(fig, update, fargs=(X, Y,), frames=range(0, len(t_euler)),
#                         interval=intervalo, blit=True, init_func=init)

#X1 = [x for x, _ in t_analitica]
#Y1 = [y for _, y in t_analitica]
#plt.plot(X1, Y1, color='r', linestyle='dotted', label='Trajetória-Analitica')
#X2 = [x for x, _ in t_euler]
#Y2 = [y for _, y in t_euler]
#plt.plot(X2, Y2, color='b', linestyle='dotted', label='Trajetória-Euler')
# plt.legend()

# rc('animation', html='jshtml')
# animacao_"alguma_coisa"

In [ ]:
# Codigo para a simulação do disparo de um projétil

d0 = [0, 0] # Posicão inicial da particula
vi = 10     # Velocidade inicial da particula
v0 = [vi*np.cos(THETA), vi*np.sin(THETA)]
a = [0, G]
v = v0
s = d0

def calcula(oldPos, oldVel, deltaT):
    newPos = []
    newVel = oldVel-G*deltaT
    v = [v0[0], newVel]
    newPos.append(oldPos[0]+v0[0]*deltaT)
    newPos.append(oldPos[1]+newVel*deltaT)
    s = newPos
    return newPos, newVel

def trajetoria_analitica():
    trajectory = []
    t = 0
    
    file = open('proj_analitica.csv', 'w+',
             newline ='')
    header = ["Aceleracao x", "Aceleracao y",
          "Velocidade x", "Velocidade y",
          "Posicao x", "Posicao y", "Tempo"]
    writer = csv.DictWriter(file, fieldnames=header, delimiter=',')
    writer.writeheader()
    
    for i in range(LIMIT):
        x = d0[0] + v0[0] * t
        y = d0[1] + v0[1] * t - G/2 * t**2
        
        writer.writerow({"Aceleracao x" : a[0],
                 "Aceleracao y" : a[1],
                 "Velocidade x" : v0[0],
                 "Velocidade y" : v0[1] - G * t,
                 "Posicao x"    : x,
                 "Posicao y"    : y,
                 "Tempo"        : t       })
        
        trajectory.append((x, y))
        t += dt
    return trajectory

def main():
        
    trajetoria = []
    atualPos = d0
    atualVel = v0[1]
    file = open('proj.csv', 'w+',
             newline ='')
    header = ["Aceleracao x", "Aceleracao y",
          "Velocidade x", "Velocidade y",
          "Posicao x", "Posicao y", "Tempo"]
    writer = csv.DictWriter(file, fieldnames=header, delimiter=',')
    writer.writeheader()
    for i in range(LIMIT):
        if atualPos[1] < 0:
            break
        writer.writerow({"Aceleracao x" : a[0],
                 "Aceleracao y" : a[1],
                 "Velocidade x" : v[0],
                 "Velocidade y" : v[1],
                 "Posicao x"    : s[0],
                 "Posicao y"    : s[1],
                 "Tempo"        : 0+i*dt  })
        trajetoria.append((atualPos[0], atualPos[1]))
        atualPos, atualVel = calcula(atualPos, atualVel, dt)
    
    return trajetoria

# Faz a simulação e a animação
t_euler = main()
t_analitica = trajetoria_analitica()
range_x = (0, 15)
range_y = (0, 4)


fig, ax, line, line2, text_template, text = setUp(range_x, range_y, "Simulação Projétil")
    
intervalo = dt*LIMIT
animacao_projetil = animation.FuncAnimation(fig, update, fargs=(t_euler, t_analitica,), frames=range(0, len(t_euler)),
                        interval=intervalo, blit=True, init_func=init)

X1 = [x for x, _ in t_analitica]
Y1 = [y for _, y in t_analitica]
plt.plot(X1, Y1, color='r', linestyle='dotted', label='Trajetória-Analitica')
X2 = [x for x, _ in t_euler]
Y2 = [y for _, y in t_euler]
plt.plot(X2, Y2, color='b', linestyle='dotted', label='Trajetória-Euler')
plt.legend()

rc('animation', html='jshtml')
animacao_projetil

In [ ]:
# Código para a simulação de uma queda em rampa

d0 = [0, 10]
v0 = [0, 0]
theta = np.pi/4
g0 = [G*np.sin(theta), G*np.cos(THETA)]
a = g0
v = v0
s = d0

def calcula(oldPos, oldVel, deltaT):
    newPos = []
    newVel = []
    newVel.append(oldVel[0]+g0[0]*deltaT)
    newVel.append(oldVel[1]-g0[1]*deltaT)
    v = newVel
    newPos.append(oldPos[0]+newVel[0]*deltaT)
    newPos.append(oldPos[1]+newVel[1]*deltaT)
    s = newPos
    return newPos, newVel

def trajetoria_analitica():
    trajectory = []
    t = 0
    
    file = open('rampa_analitica.csv', 'w+',
             newline ='')
    header = ["Aceleracao x", "Aceleracao y",
          "Velocidade x", "Velocidade y",
          "Posicao x", "Posicao y", "Tempo"]
    writer = csv.DictWriter(file, fieldnames=header, delimiter=',')
    writer.writeheader()
     
    for i in range(LIMIT):
        x = d0[0] + v0[0] * t + g0[0] / 2 * t ** 2
        y = d0[1] + v0[1] * t + g0[1] / 2 * t ** 2
        
        v[0] = v0[0] + g0[0] * t
        v[1] = v0[1] + g0[1] * t
        
        writer.writerow({"Aceleracao x" : a[0],
                 "Aceleracao y" : a[1],
                 "Velocidade x" : v[0],
                 "Velocidade y" : v[1],
                 "Posicao x"    : x,
                 "Posicao y"    : y,
                 "Tempo"        : t       })
        
        trajectory.append((x, y))
        t += dt
    return trajectory

def main():
    trajetoria = []
    atualPos = d0
    atualVel = v0
    file = open('rampa.csv', 'w+',
             newline ='')
    header = ["Aceleracao x", "Aceleracao y",
          "Velocidade x", "Velocidade y",
          "Posicao x", "Posicao y", "Tempo"]
    writer = csv.DictWriter(file, fieldnames=header, delimiter=',')
    writer.writeheader()
    for i in range(LIMIT):
        if atualPos[1] < 0:
            break
        writer.writerow({"Aceleracao x" : a[0],
                 "Aceleracao y" : a[1],
                 "Velocidade x" : v[0],
                 "Velocidade y" : v[1],
                 "Posicao x"    : s[0],
                 "Posicao y"    : s[1],
                 "Tempo"        : 0+i*dt  })
        trajetoria.append((atualPos[0], atualPos[1]))
        atualPos, atualVel = calcula(atualPos, atualVel, dt)
        
    return trajetoria

t_euler = main()
t_analitica = trajetoria_analitica()
range_x = (0, 10)
range_y = (0, 10)

fig, ax, line, line2, text_template, text = setUp(range_x, range_y, "Simulação Rampa")

intervalo = dt*LIMIT
animacao_rampa = animation.FuncAnimation(fig, update, fargs=(t_euler, t_analitica,), frames=range(0, len(t_euler)),
                        interval=intervalo, blit=True, init_func=init)

X1 = [x for x, _ in t_analitica]
Y1 = [y for _, y in t_analitica]
plt.plot(X1, Y1, color='r', linestyle='dotted', label='Trajetória-Analitica')
X2 = [x for x, _ in t_euler]
Y2 = [y for _, y in t_euler]
plt.plot(X2, Y2, color='b', linestyle='dotted', label='Trajetória-Euler')
plt.legend()

rc('animation', html='jshtml')
animacao_rampa

In [ ]:
# Código para a simulação do movimento uniformemente variado de uma partícula

d0 = [0, 0]
v0 = [10, 0]
a0 = [-3, 8]
a = a0
v = v0
s = d0

def calcula(oldPos, oldVel, deltaT):
    global v, s
    newPos = []
    newVel = []
    newVel.append(oldVel[0]+a0[0]*dt)
    newVel.append(oldVel[1]+a0[1]*dt)
    v = newVel
    newPos.append(oldPos[0]+newVel[0]*dt)
    newPos.append(oldPos[1]+newVel[1]*dt)
    s = newPos
    return newPos, newVel

def trajetoria_analitica():
    trajectory = []
    t = 0
    
    file = open('muv_analitica.csv', 'w+',
             newline ='')
    header = ["Aceleracao x", "Aceleracao y",
          "Velocidade x", "Velocidade y",
          "Posicao x", "Posicao y", "Tempo"]
    writer = csv.DictWriter(file, fieldnames=header, delimiter=',')
    writer.writeheader()

    for i in range(LIMIT):
        x = d0[0] + v0[0] * t + a0[0] / 2 * t ** 2
        y = d0[1] + v0[1] * t + a0[1] / 2 * t ** 2
        
        v[0] = v0[0] + a0[0] * t
        v[1] = v0[1] + a0[1] * t

        writer.writerow({"Aceleracao x" : a[0],
                 "Aceleracao y" : a[1],
                 "Velocidade x" : v[0],
                 "Velocidade y" : v[1],
                 "Posicao x"    : x,
                 "Posicao y"    : y,
                 "Tempo"        : t  })

        trajectory.append((x, y))
        t += dt
    return trajectory

def main():
      
    trajetoria = []
    atualPos = d0
    atualVel = v0
    file = open('muv.csv', 'w+',
             newline ='')
    header = ["Aceleracao x", "Aceleracao y",
          "Velocidade x", "Velocidade y",
          "Posicao x", "Posicao y", "Tempo"]
    writer = csv.DictWriter(file, fieldnames=header, delimiter=',')
    writer.writeheader()

    with file:
        for i in range(LIMIT):
        
            writer.writerow({"Aceleracao x" : a[0],
                 "Aceleracao y" : a[1],
                 "Velocidade x" : v[0],
                 "Velocidade y" : v[1],
                 "Posicao x"    : s[0],
                 "Posicao y"    : s[1],
                 "Tempo"        : 0+i*dt  })
            trajetoria.append((atualPos[0], atualPos[1]))
            atualPos, atualVel = calcula(atualPos, atualVel, dt)
        
    return trajetoria

t_euler = main()
t_analitica = trajetoria_analitica()
range_x = (0, 30)
range_y = (0, 100)
    
fig, ax, line, line2, text_template, text = setUp(range_x, range_y, "Simulação Movimento Uniformemente Variado")

intervalo = dt*LIMIT/100
animacao_muv = animation.FuncAnimation(fig, update, fargs=(t_euler, t_analitica,), frames=range(0, len(t_euler)),
                        interval=intervalo, blit=True, init_func=init)


X1 = [x for x, _ in t_analitica]
Y1 = [y for _, y in t_analitica]
plt.plot(X1, Y1, color='r', linestyle='dotted', label='Trajetória-Analitica')
X2 = [x for x, _ in t_euler]
Y2 = [y for _, y in t_euler]
plt.plot(X2, Y2, color='b', linestyle='dotted', label='Trajetória-Euler')
plt.legend()

rc('animation', html='jshtml')
animacao_muv

In [ ]:
# Código para a simulação do movimento circular de uma partícula

theta0 = 0
omega = 0.5

def calcula(oldTheta):
    theta = oldTheta + omega * dt
    return theta

def trajetoria_analitica():
    trajectory = []
    t = 0
    theta = theta0
    
    file = open('circular_analitica.csv', 'w+', newline ='')
    header = ["Velocidade angular", "Posicao x", "Posicao y", "Tempo"]
    writer = csv.DictWriter(file, fieldnames=header, delimiter=',')
    writer.writeheader()
    
    for i in range(LIMIT):
        
        writer.writerow({"Velocidade angular" : omega,
                 "Posicao x"    : np.sin(theta),
                 "Posicao y"    : np.cos(theta),
                 "Tempo"        : t  })

        trajectory.append((np.sin(theta), np.cos(theta)))
        t += dt
        theta = theta0 + omega * t
    
    return trajectory

def main():
    trajetoria = []
    theta = theta0
    
    file = open('circular.csv', 'w+', newline ='')
    header = ["Velocidade angular", "Posicao x", "Posicao y", "Tempo"]
    writer = csv.DictWriter(file, fieldnames=header, delimiter=',')
    writer.writeheader()
    
    for i in range(LIMIT):
        trajetoria.append((np.sin(theta), np.cos(theta)))
        
        writer.writerow({"Velocidade angular" : omega,
                 "Posicao x"    : np.sin(theta),
                 "Posicao y"    : np.cos(theta),
                 "Tempo"        : 0+i*dt  })
        
        theta = calcula(theta)
    #return [np.sin(t) for t in Theta], [np.cos(t) for t in Theta]
    return trajetoria

t_euler = main()
t_analitica = trajetoria_analitica()
range_x = (-1.5, 1.5)
range_y = (-1.5, 1.5)

fig, ax, line, line2, text_template, text = setUp(range_x, range_y, "Rotação")
    
intervalo = dt*LIMIT
animacao_rotacao = animation.FuncAnimation(fig, update, fargs=(t_euler, t_analitica,), frames=range(0, len(t_euler)),
                        interval=intervalo, blit=True, init_func=init)

X1 = [x for x, _ in t_analitica]
Y1 = [y for _, y in t_analitica]
plt.plot(X1, Y1, color='r', linestyle='dotted', label='Trajetória-Analitica')
X2 = [x for x, _ in t_euler]
Y2 = [y for _, y in t_euler]
plt.plot(X2, Y2, color='b', linestyle='dotted', label='Trajetória-Euler')
plt.legend()

rc('animation', html='jshtml')
animacao_rotacao

In [ ]:
def computa_erro_abs(tab1, tab2, titulo = ""):
    '''
    Calcula o erro absoluto entre duas tabelas
    '''
    
    df_tab1 = pd.read_csv(tab1)
    df_tab2 = pd.read_csv(tab2)

    df_tempo = df_tab1.loc[:, "Tempo"]

    colunas = list(df_tab1.columns)
    colunas.remove("Tempo")

    # Calcula o erro absoluto entre as duas tabelas
    err_table = abs(df_tab1.loc[:, colunas] - df_tab2.loc[:, colunas])

    # Junta com o tempo
    err_table = err_table.join(df_tempo)

    # Renomeia as colunas para melhorar a legibilidade
    err_table = err_table.rename(columns = {i : "Erro " + i for i in colunas})

    if (titulo != ""):
        err_table.to_csv(titulo)
        
    return err_table

In [ ]:
computa_erro_abs("muv.csv", "muv_analitica.csv", "muv_err.csv")

In [ ]:
computa_erro_abs("rampa.csv", "rampa_analitica.csv", "rampa_err.csv")

In [ ]:
computa_erro_abs("proj.csv", "proj_analitica.csv", "proj_err.csv")

In [ ]:
computa_erro_abs("circular.csv", "circular_analitica.csv", "circular_err.csv")